In [9]:
import pandas as pd;

df = pd.read_csv('data.csv');
print(df.head())
print(df.shape)

   Unnamed: 0  count  hate_speech  offensive_language  neither  class   
0           0      3            0                   0        3      2  \
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  
(24783, 7)


In [23]:
import re, string
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jainh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jainh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jainh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jainh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
wl = WordNetLemmatizer()

def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string))
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)]
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [25]:
df['clean_text'] = df['tweet'].apply(lambda x: finalpreprocess(x))

In [29]:
df['clean_text_tok']=[nltk.word_tokenize(i) for i in df['clean_text']]

model = Word2Vec(df['clean_text_tok'],min_count=1)  

w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))

In [30]:
for i in df['clean_text']:
    print(i)

rt mayasolovely woman complain clean house amp man always take trash
rt mleew boy dat cold tyga dwn bad cuffin dat hoe st place
rt urkindofbrand dawg rt sbaby life ever fuck bitch start cry confuse shit
rt c g anderson viva base look like tranny
rt shenikaroberts shit hear might true might faker bitch tell ya
madison x shit blow claim faithful somebody still fuck hoe
brighterdays sit hate another bitch get much shit go
selfiequeenbri cause tire big bitch come u skinny girl
amp might get ya bitch back amp thats
rhythmixx hobby include fight mariam bitch
keeks bitch curve everyone lol walk conversation like smh
murda gang bitch gang land
hoe smoke loser yea go ig
bad bitch thing like
bitch get
bitch nigga miss
bitch plz whatever
bitch love
bitch get cut everyday b
black bottle amp bad bitch
break bitch cant tell nothing
cancel bitch like nino
cant see hoe wont change
fuck bitch dont even suck dick kermit video bout fuck ig
get ya bitch tip toe hardwood floor http co cou wq l q
pussy lip 

In [50]:
train, test = train_test_split(df,test_size=0.2,shuffle=True)

In [35]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline

In [39]:
LogReg_pipeline = Pipeline([('tfidf', TfidfVectorizer()), ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1))])

In [61]:
categories = ['hate', 'offensive', 'neither']

def comp(x,y):
    if(x==y):
        return 1.0
    else:
        return 0.0

train['hate'] = train['class'].apply(lambda x: comp(x,0))
train['offensive'] = train['class'].apply(lambda x: comp(x,1))
train['neither'] = train['class'].apply(lambda x: comp(x,2))
test['hate'] = test['class'].apply(lambda x: comp(x,0))
test['offensive'] = test['class'].apply(lambda x: comp(x,1))
test['neither'] = test['class'].apply(lambda x: comp(x,2))


for category in categories:
    print('... Processing {}'.format(category))
    LogReg_pipeline.fit(train.clean_text, train[category])
    prediction = LogReg_pipeline.predict(test.clean_text)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing hate
Test accuracy is 0.9435142223118822
... Processing offensive
Test accuracy is 0.9007464192051644
... Processing neither
Test accuracy is 0.9447246318337704


In [56]:
from sklearn.naive_bayes import MultinomialNB

In [58]:
NB_pipeline =  Pipeline([('tfidf', TfidfVectorizer()), ('clf', OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None)))])

In [62]:
for category in categories:
    print('... Processing {}'.format(category))
    NB_pipeline.fit(train.clean_text, train[category])
    prediction = NB_pipeline.predict(test.clean_text)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing hate
Test accuracy is 0.9414968731087351
... Processing offensive
Test accuracy is 0.795844260641517
... Processing neither
Test accuracy is 0.8369981843857172


In [63]:
from sklearn.svm import LinearSVC

In [64]:
SVC_pipeline =  Pipeline([('tfidf', TfidfVectorizer()), ('clf', OneVsRestClassifier(LinearSVC()))])

In [69]:
for category in categories:
    print('... Processing {}'.format(category))
    SVC_pipeline.fit(train.clean_text, train[category])
    prediction = SVC_pipeline.predict(["Fuck You Stupid Ass"])
    print('{} : {}'.format(category, prediction))
    # print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing hate
hate : [0.]
... Processing offensive
offensive : [1.]
... Processing neither
neither : [0.]
